# Financial Distribution Company - Machine Learning Solution

### Build a classifier to predict if a financial agent will get business

   - Defining the Problem statement
   - Collecting the data
   - Exploratory data analysis
   - Feature Engineering
   - Modelling
   - Predicting
   - Evaluation
   - Prepare Submission File

### Defining the Problem statement
Your client is a Financial Distribution company. Over the last 10 years, they have created an offline distribution channel across country. They sell Financial products to consumers by hiring agents in their network. These agents are freelancers and get commission when they make a product sale.

**Overview of your client On-boarding process**
The Managers at your client are primarily responsible for recruiting agents. Once a manager has identified a potential applicant, the would explain the business opportunity to the agent. Once the agent provides the consent, an application is made to your client to become an agent. This date is known as application_receipt_date. In the next 3 months, this potential agent has to undergo a 7 days training at the your client's branch (about Sales processes and various products) and clear a subsequent examination in order to become an agent.

**The problem - Who are the best agents?**
As is obvious in the above process, there is a significant investment which your cleint makes in identifying, training and recruiting these agents. However, there are a set of agents who do not bring in the expected resultant business. Your client is looking for help from data scientists like you to help them provide insigths using their past recruitment data. They want to predict the target variable for each potential agent, which would help them identify the right agents to hire.

**Key Points** : 
- It has data for period Apr'2007 to Jan'2009 (For Jan'09 only 99 records for 01-Jan-09)
- The training data for period Apr'2007 to 01-Jul-2008
- Public leaderboard is based on First 2 months of the test dataset (02-Jul-2008 and 01-Sep-2008) and rest of test dataset is used for Private leaderboard
- Evaluation Metric is ROC - AUC. For more info, check here
- You are expected to upload the solution in the format of "sample_submission.csv".

**Data Definitions**
1. ID - Unique Application ID \n
2. Office_PIN - PINCODE of Your client's Offices
3. Application_Receipt_Date - Date of Application
4. Applicant_City_PIN - PINCODE of Applicant Address
5. Applicant_Gender - Applicant's Gender
6. Applicant_BirthDate - Applicant's Birthdate
7. Applicant_Marital_Status - Applicant's Marital Status
8. Applicant_Occupation - Applicant's Occupation
9. Applicant_Qualification - Applicant's Educational Qualification
10. Manager_DOJ - Manager's Date of Joining
11. Manager_Joining_Designation - Manager's Joining Designation
12. Manager_Current_Designation - Manager's Designation at the time of application sourcing
13. Manager_Grade - Manager's Grade
14. Manager_Status - Current Employment Status (Probation / Confirmation)
15. Manager_Gender - Manager's Gender
16. Manager_DoB - Manager's Birthdate
17. Manager_Num_Application - No. of Applications sourced in last 3 months by the Manager
18. Manager_Num_Coded - No. of agents recruited by the manager in last 3 months
19. Manager_Business - Amount of business sourced by the manager in last 3 months
20. Manager_Num_Products - Number of products sold by the manager in last 3 months
21. Manager_Business2 - Amount of business sourced by the manager in last 3 months excluding business from their Category A advisor
22. Manager_Num_Products2 - Number of products sold by the manager in last 3 months excluding business from their Category A advisor
23. Business_Sourced(Target) - Business sourced by applicant within 3 months [1/0] of recruitment

### Load Libraries

In [92]:
import pandas as pd
import numpy as np
import os.path
from os import path
from sklearn import model_selection
from sklearn import ensemble
from sklearn import preprocessing
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import joblib

### Read Data

In [93]:
dftrain = pd.read_csv(f"Data/Train_pjb2QcD.csv")
dftest = pd.read_csv(f"Data/Test_wyCirpO.csv")

*Let's get some intuition on the data*

In [94]:
dftrain.shape

(9527, 23)

In [95]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9527 entries, 0 to 9526
Data columns (total 23 columns):
ID                             9527 non-null object
Office_PIN                     9527 non-null int64
Application_Receipt_Date       9527 non-null object
Applicant_City_PIN             9430 non-null float64
Applicant_Gender               9460 non-null object
Applicant_BirthDate            9454 non-null object
Applicant_Marital_Status       9454 non-null object
Applicant_Occupation           8306 non-null object
Applicant_Qualification        9441 non-null object
Manager_DOJ                    8844 non-null object
Manager_Joining_Designation    8844 non-null object
Manager_Current_Designation    8844 non-null object
Manager_Grade                  8844 non-null float64
Manager_Status                 8844 non-null object
Manager_Gender                 8844 non-null object
Manager_DoB                    8844 non-null object
Manager_Num_Application        8844 non-null float64
Manager

In [96]:
dftrain.isnull().sum()

ID                                0
Office_PIN                        0
Application_Receipt_Date          0
Applicant_City_PIN               97
Applicant_Gender                 67
Applicant_BirthDate              73
Applicant_Marital_Status         73
Applicant_Occupation           1221
Applicant_Qualification          86
Manager_DOJ                     683
Manager_Joining_Designation     683
Manager_Current_Designation     683
Manager_Grade                   683
Manager_Status                  683
Manager_Gender                  683
Manager_DoB                     683
Manager_Num_Application         683
Manager_Num_Coded               683
Manager_Business                683
Manager_Num_Products            683
Manager_Business2               683
Manager_Num_Products2           683
Business_Sourced                  0
dtype: int64

In [97]:
dftest.shape

(5045, 22)

In [98]:
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5045 entries, 0 to 5044
Data columns (total 22 columns):
ID                             5045 non-null object
Office_PIN                     5045 non-null int64
Application_Receipt_Date       5045 non-null object
Applicant_City_PIN             5011 non-null float64
Applicant_Gender               5023 non-null object
Applicant_BirthDate            5022 non-null object
Applicant_Marital_Status       5015 non-null object
Applicant_Occupation           4041 non-null object
Applicant_Qualification        5001 non-null object
Manager_DOJ                    4221 non-null object
Manager_Joining_Designation    4221 non-null object
Manager_Current_Designation    4221 non-null object
Manager_Grade                  4221 non-null float64
Manager_Status                 4221 non-null object
Manager_Gender                 4221 non-null object
Manager_DoB                    4221 non-null object
Manager_Num_Application        4221 non-null float64
Manager

In [99]:
dftrain.describe()

,Office_PIN,Applicant_City_PIN,Manager_Grade,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2,Business_Sourced
count,9527.000000,9430.000000,8844.000000,8844.000000,8844.000000,8.844000e+03,8844.000000,8.844000e+03,8844.000000,9527.00000
mean,452894.372205,456784.547296,3.264134,1.939733,0.758933,1.843710e+05,7.152307,1.829263e+05,7.131275,0.34292
std,235690.618252,239291.081207,1.137449,2.150529,1.188644,2.747163e+05,8.439351,2.718021e+05,8.423597,0.47471
min,110005.000000,110001.000000,1.000000,0.000000,0.000000,-2.652890e+05,0.000000,-2.652890e+05,0.000000,0.00000
25%,226001.000000,226020.000000,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.00000
50%,416001.000000,422005.500000,3.000000,1.000000,0.000000,1.021780e+05,5.000000,1.017140e+05,5.000000,0.00000
75%,695014.000000,695017.000000,4.000000,3.000000,1.000000,2.471165e+05,11.000000,2.464612e+05,11.000000,1.00000
max,851101.000000,995657.000000,10.000000,22.000000,9.000000,3.578265e+06,101.000000,3.578265e+06,101.000000,1.00000


In [100]:
dftest.describe()

,Office_PIN,Applicant_City_PIN,Manager_Grade,Manager_Num_Application,Manager_Num_Coded,Manager_Business,Manager_Num_Products,Manager_Business2,Manager_Num_Products2
count,5045.000000,5011.000000,4221.000000,4221.000000,4221.000000,4.221000e+03,4221.000000,4.221000e+03,4221.000000
mean,431478.138751,433049.851527,3.352760,2.521204,1.104951,1.115425e+05,3.864250,9.895073e+04,3.495143
std,230425.779147,231655.574011,0.994504,2.491566,1.473985,1.640698e+05,4.688678,1.469193e+05,4.241165
min,110005.000000,110001.000000,2.000000,0.000000,0.000000,-3.196900e+04,0.000000,-3.196900e+04,0.000000
25%,221010.000000,224141.000000,3.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
50%,400705.000000,411005.000000,3.000000,2.000000,1.000000,5.002600e+04,2.000000,4.653700e+04,2.000000
75%,641018.000000,641002.000000,4.000000,4.000000,2.000000,1.757620e+05,7.000000,1.537820e+05,6.000000
max,851101.000000,884112.000000,8.000000,17.000000,10.000000,2.196480e+06,31.000000,2.196480e+06,30.000000


Combine the train and test data

In [101]:
train_test_data = [dftrain, dftest]

### Data Cleansing and Feature Engineering

In [102]:
for dataset in train_test_data:
        dataset['Application_Receipt_Date'] = pd.to_datetime(dataset['Application_Receipt_Date']).dt.strftime(f"%d/%m/%Y")
        dataset['Applicant_BirthDate'] = pd.to_datetime(dataset['Applicant_BirthDate']).dt.strftime(f"%d/%m/%Y")
        dataset['Manager_DoB'] = pd.to_datetime(dataset['Manager_DoB']).dt.strftime(f"%d/%m/%Y")            
       
        
#         dftrain.to_csv("input/train_tofill.csv", index=False)
#         dftest.to_csv("input/test_tofill.csv", index=False)

### Data Imputation

#### Fill NA Technique

In [103]:
# dftrain.fillna("-9999999", inplace=True)
# dftest.fillna("-9999999", inplace=True)
# dftrain.to_csv(f"Data/train_fillna_IDSC.csv", index=False)
# dftest.to_csv(f"Data/test_fillna_IDSC.csv", index=False)

#### Mean / Median Technique

In [104]:
numeric_cols = ['Manager_Num_Application','Manager_Num_Coded','Manager_Business','Manager_Num_Products','Manager_Business2','Manager_Num_Products2']

In [105]:
for df in train_test_data:
    df['Manager_Num_Application'].fillna(df.Manager_Num_Application.mean(), inplace=True)
    df['Manager_Num_Coded'].fillna(df.Manager_Num_Coded.mean(), inplace=True)
    df['Manager_Business'].fillna(df.Manager_Business.mean(), inplace=True)
    df['Manager_Num_Products'].fillna(df.Manager_Num_Products.mean(), inplace=True)
    df['Manager_Business2'].fillna(df.Manager_Business2.mean(), inplace=True)
    df['Manager_Num_Products2'].fillna(df.Manager_Num_Products2.mean(), inplace=True)  

In [106]:
# for df in train_test_data:
#     df['Office_PIN'].fillna(df.Office_PIN.mode(), inplace=True)
#     df['Application_Receipt_Date'].fillna(df.Application_Receipt_Date.mode(), inplace=True)
#     df['Applicant_City_PIN'].fillna(df.Applicant_City_PIN.mode(), inplace=True)
#     df['Applicant_Gender'].fillna(df.Applicant_Gender.mode(), inplace=True)
#     df['Applicant_BirthDate'].fillna(df.Applicant_BirthDate.mode(), inplace=True)
#     df['Applicant_Marital_Status'].fillna(df.Applicant_Marital_Status.mode(), inplace=True)
#     df['Applicant_Occupation'].fillna(df.Applicant_Occupation.mode(), inplace=True)
#     df['Applicant_Qualification'].fillna(df.Applicant_Qualification.mode(), inplace=True)
#     df['Manager_DOJ'].fillna(df.Manager_DOJ.mode(), inplace=True)
#     df['Manager_Joining_Designation'].fillna(df.Manager_Joining_Designation.mode(), inplace=True)
#     df['Manager_Current_Designation'].fillna(df.Manager_Current_Designation.mode(), inplace=True)
#     df['Manager_Grade'].fillna(df.Manager_Grade.mode(), inplace=True)
#     df['Manager_Status'].fillna(df.Manager_Status.mode(), inplace=True)
#     df['Manager_Gender'].fillna(df.Manager_Gender.mode(), inplace=True)
#     df['Manager_DoB'].fillna(df.Manager_DoB.mode(), inplace=True)

In [107]:
for dataset in train_test_data:
    dataset['App_Year'] = pd.DatetimeIndex(dataset['Application_Receipt_Date']).year
    dataset['App_Month'] = pd.DatetimeIndex(dataset['Application_Receipt_Date']).month
    dataset["IsApplicantFemale?"] = np.where(dataset['Applicant_Gender']=='F', 1, 0)
    dataset["IsManagerFemale?"] = np.where(dataset['Manager_Gender']=='F', 1, 0)
    dataset['Manager_Success_Rate'] = np.where((dataset['Manager_Num_Application'].isnull() | dataset['Manager_Num_Application']==0),
                                        np.NaN,
                                        (dataset.Manager_Num_Coded/dataset.Manager_Num_Application)*100)
    dataset['Applicant_Age'] = pd.DatetimeIndex(dataset['Application_Receipt_Date']).year - pd.DatetimeIndex(dataset['Applicant_BirthDate']).year
    dataset['Manager_Age'] = pd.DatetimeIndex(dataset['Application_Receipt_Date']).year - pd.DatetimeIndex(dataset['Manager_DoB']).year

In [108]:
dftrain.fillna("-9999999", inplace=True)
dftest.fillna("-9999999", inplace=True)
# dftrain.to_csv(f"Data/train_fillna_IDSC.csv", index=False)
# dftest.to_csv(f"Data/test_fillna_IDSC.csv", index=False)

In [110]:
dftrain.isnull().sum()

ID                             0
Office_PIN                     0
Application_Receipt_Date       0
Applicant_City_PIN             0
Applicant_Gender               0
Applicant_BirthDate            0
Applicant_Marital_Status       0
Applicant_Occupation           0
Applicant_Qualification        0
Manager_DOJ                    0
Manager_Joining_Designation    0
Manager_Current_Designation    0
Manager_Grade                  0
Manager_Status                 0
Manager_Gender                 0
Manager_DoB                    0
Manager_Num_Application        0
Manager_Num_Coded              0
Manager_Business               0
Manager_Num_Products           0
Manager_Business2              0
Manager_Num_Products2          0
Business_Sourced               0
App_Year                       0
App_Month                      0
IsApplicantFemale?             0
IsManagerFemale?               0
Manager_Success_Rate           0
Applicant_Age                  0
Manager_Age                    0
dtype: int

### Split the data into K folds

In [111]:
df = pd.read_csv(f"Data/train_fillna_IDSC.csv")

df["kfold"]=-1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df, y=df.Business_Sourced.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold

df.to_csv(f"Data/train_IDSC_folds.csv", index=False)

7621 1906
7621 1906
7622 1905
7622 1905
7622 1905


### Modelling and Evaluation

In [112]:
df = pd.read_csv(f"Data/train_IDSC_folds.csv")
df_test = pd.read_csv(f"Data/test_fillna_IDSC.csv")
    
MODEL = 'XGBoost'
FOLD_MAPPING = { \
    0: [1,2,3,4],
    1: [0,2,3,4],
    2: [0,1,3,4],
    3: [0,1,2,4],
    4: [0,1,2,3]
}

# FOLD_MAPPING = { \
#     0: [1,2,3,4,5,6,7,8,9],
#     1: [0,2,3,4,5,6,7,8,9],
#     2: [0,1,3,4,5,6,7,8,9],
#     3: [0,1,2,4,5,6,7,8,9],
#     4: [0,1,2,3,5,6,7,8,9],
#     5: [0,1,2,3,4,6,7,8,9],
#     6: [0,1,2,3,4,5,7,8,9],
#     7: [0,1,2,3,4,5,6,8,9],
#     8: [0,1,2,3,4,5,6,7,9],
#     9: [0,1,2,3,4,5,6,7,8]
# }

for FOLD in range(5):
    train_df = df[df.kfold.isin(FOLD_MAPPING.get(int(FOLD)))].reset_index(drop=True)
    valid_df = df[df.kfold==FOLD].reset_index(drop=True)

    ytrain = train_df.Business_Sourced.values
    yvalid = valid_df.Business_Sourced.values

    train_df = train_df.drop(["ID","Business_Sourced","kfold"], axis=1)
    valid_df = valid_df.drop(["ID","Business_Sourced","kfold"], axis=1)

    valid_df = valid_df[train_df.columns]

    label_encoders = {}

    for c in train_df.columns:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(train_df[c].values.tolist() + valid_df[c].values.tolist() + df_test[c].values.tolist())
        train_df.loc[:,c] = lbl.transform(train_df[c].values.tolist())
        valid_df.loc[:,c] = lbl.transform(valid_df[c].values.tolist())
        label_encoders[c] = lbl


    #data is ready to train
    clf = XGBClassifier(max_depth=3,learning_rate=0.1,n_estimators=500,verbosity=1,objective='binary:logistic',booster='gbtree',n_jobs=1)
    clf.fit(train_df, ytrain)
    preds = clf.predict_proba(valid_df)[:,1]
    print(metrics.roc_auc_score(yvalid, preds))

    joblib.dump(label_encoders, f"models/{MODEL}_{FOLD}_label_encoder.pkl")
    joblib.dump(clf, f"models/{MODEL}_{FOLD}.pkl")
    joblib.dump(train_df.columns, f"models/{MODEL}_{FOLD}_columns.pkl")

0.6634217056990162
0.6241744096296079
0.6189043930935618
0.6313965037257387
0.6197288014521329


### Predicting

In [113]:
def predict():
    df = pd.read_csv(f"Data/test_fillna_IDSC.csv")
    
    test_idx = df["ID"].values
    predictions = None
     
    for FOLD in range(5):
        print(FOLD)
        df = pd.read_csv(f"Data/test_fillna_IDSC.csv")
        # cols = [c for c in df_TEST.columns if c not in ["id", "target"]]
    
        # for c in cols:
        #     df_TEST.loc[:, c] = df_TEST.loc[:, c].fillna("-9999999")

            
        # df = df_TEST.copy(deep=True)

        
        encoders = joblib.load(os.path.join("models", f"{MODEL}_{FOLD}_label_encoder.pkl"))
        cols = joblib.load(os.path.join("models", f"{MODEL}_{FOLD}_columns.pkl"))

        for c in encoders:
            #print(c)
            lbl = encoders[c]
            df.loc[:,c] = lbl.transform(df[c].values.tolist())
            

        #data is ready to train
        clf = joblib.load(os.path.join("models", f"{MODEL}_{FOLD}.pkl"))
        
        df = df[cols]
        preds = clf.predict_proba(df)[:,1]
        
        if FOLD==0:
            predictions = preds
        else:
            predictions += preds

    predictions /= 5

    sub = pd.DataFrame(np.column_stack((test_idx, predictions)), columns=["ID", "Business_Sourced"])
    return sub

### Prepare submission file

In [114]:
submission = predict()
submission.to_csv(f"models/{MODEL}.csv", index=False)

0
1
2
3
4
